# Experimentation Notebook on using Langchain

In this notebook I ask myself a couple of questions that I want to answer and address regarding the prompto chatbot for the pronto framework build on Godot.

## What do I want to try?

[x] Generate documentation of pronto code
[ ] Use generated docs to answer questions
[ ] (optional) Link Readme to generated docs

## Generating pronto documentation

https://github.com/GDQuest/gdscript-docs-maker

I used this tool to first build a reference json file and generate afterwards markdown files of each behavior.

I had a couple of problems with the tool:
1. The automated generate reference shell script didn't work for me. For some reason the Editor just crashed.
2. A workaround included importing the required GDScripts directly into Godot and let it generate from the Editor. One additional step I had to take was to specify only the directory with behavior classes (res://addons/pronto/behavior) as one or two directory levels higher crashed the editor (also happened in a different Godot project).
3. For the actual markdown generation a Python module called gdscript_docs_maker is used. For some reason trying to use the module directly didn't work, but calling the `__main__` script with it does. I created an issue on GitHub for this: https://github.com/GDQuest/gdscript-docs-maker/issues/94

The resulting markdown files are saved in the folder *pronto-docs*.

## Prerequisites

Below are the necessary code blocks to get started:
1. Install all Python modules for development
2. Load the environment variables for OpenAI, etc.
3. Clone the pronto repo

In [15]:
!pip install langchain openai GitPython chromadb unstructured markdown sentence_transformers tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.6/955.6 kB 5.1 MB/s eta 0:00:0000:0100:01m


In [12]:
from dotenv import load_dotenv
load_dotenv()

True

## Question 1: Retrieve behavior details from general readme documentation

*What are you trying?*
Use the newly generated pronto documentation together with the readme to obtain better retrieving.

*How do you approach it?*
As the readme contains all behaviors an agent can look up the details if it needs to. So I would make it retrieve the important parts of the readme as well as the related behavior documentation.

*What is the desired outcome?*
1. Better results and information for the llm to generate output

In [3]:
from langchain.document_loaders import GitLoader

code_loader = GitLoader(
    clone_url="https://github.com/hpi-swa-lab/godot-pronto",
    repo_path="./pronto",
    branch="master",
    file_filter=lambda file_path: file_path.endswith(".gd")
)
code_documents = code_loader.load()

In [112]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredMarkdownLoader

readme_loader = UnstructuredMarkdownLoader("pronto/README.md")
readme_document = readme_loader.load()

docs_loader = DirectoryLoader("pronto-docs", glob="**/*.md", loader_cls=UnstructuredMarkdownLoader)
docs_document = docs_loader.load()

In [121]:
readme_document

[Document(page_content='Pronto\n\nPronto is a framework for Godot to make prototyping game mechanics faster. It is not a framework aimed at helping to create entire games faster. The resulting prototypes are a means to quickly explore ideas, throw away the prototype, and only properly implement ideas that turned out well.\n\nPlay prototypes created in pronto here.\n\nFunction\n\nThe main idea of Pronto is to make behavior visible. Our hypothesis is that it will be easier to create and tweak game mechanics. For example, instead of defining numbers in code for the distance a platform moves, we use handles in the game world to visually direct it.\n\nPronto consists of a set of Godot Nodes called Behavior that can be added to a Godot scene. These behaviors are aspects that, when combined, result in the expression of complex behavior in the Godot scene. All behaviors have visual representation in the scene and primarily function through an event system called Connections. For example, if a 

In [113]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

readme_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=400, chunk_overlap=50)
readme_splits = readme_splitter.split_documents(readme_document)
docs_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=100, chunk_overlap=20)
docs_splits = docs_splitter.split_documents(docs_document)

In [114]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [115]:
from langchain_community.vectorstores.chroma import Chroma

readme_db = Chroma.from_documents(documents=readme_splits, embedding=embeddings)
readme_retriever = readme_db.as_retriever(
    search_type="mmr",  # Also test "similarity",
    search_kwargs={"k": 2}
)

docs_db = Chroma.from_documents(documents=docs_splits, embedding=embeddings)
docs_retriever = docs_db.as_retriever(
    search_type="mmr",  # Also test "similarity",
    search_kwargs={"k": 2}
)

In [116]:
from langchain.tools.retriever import create_retriever_tool

readme_tool = create_retriever_tool(
    readme_retriever,
    "search_readme",
    "Searches and returns documentation"
)

docs_tool = create_retriever_tool(
    docs_retriever,
    "search_docs",
    "Searches documentation of behaviors, useful if you want to know more about a specific behavior"
)

tools = [docs_tool, readme_tool]

In [117]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.2)

In [118]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)

In [119]:
def ask(question):
    adversarial_prompt = f"""
You are answering prompts for a game prototyping framework called pronto which is build on top of the Godot Game Engine. Try to answer questions with the provided tools which help finding relevant information about the pronto framework.

For this new prompt, you only have access to the tools {", ".join([tool.name for tool in tools])}.

Question: {question}?"""
    return agent.run(adversarial_prompt)

In [120]:
ask("What types of connections?")



> Entering new AgentExecutor chain...
I need to find information about the types of connections in the pronto framework.
Action: search_docs
Action Input: "types of connections"
Observation: [Document(page_content='Creating\n\nConnections are created by hovering the "+" that appears below selected nodes. There are two types of connections.', metadata={'source': 'pronto/README.md'}), Document(page_content='Connections are an extension of Godot signals to be more flexible. Connections can be dragged from any behavior to any arbitrary node in a scene. They are the primary means to assemble your game by', metadata={'source': 'pronto/README.md'})]
Thought:I found two documents that mention the types of connections in the pronto framework.
Action: search_readme
Action Input: "types of connections"
Observation: [Document(page_content='Creating\n\nConnections are created by hovering the "+" that appears below selected nodes. There are two types of connections.', metadata={'source': 'pronto/R

'There are two types of connections in the pronto framework.'

In [109]:
with open("questions.txt") as f:
    questions = f.readlines()

In [110]:
for question in questions:
    ask(question)



> Entering new AgentExecutor chain...
I need to find information on how to connect behaviors in the pronto framework.
Action: search_docs
Action Input: "connect behaviors"
Observation: [Document(page_content='your game by wiring Behaviors together.', metadata={'source': 'pronto/README.md'}), Document(page_content='Creating\n\nConnections are created by hovering the "+" that appears below selected nodes. There are two types of connections.', metadata={'source': 'pronto/README.md'})]
Thought:I found some relevant information in the README.md file of the pronto framework.
Action: search_readme
Action Input: "connect behaviors"
Observation: [Document(page_content='your game by wiring Behaviors together.', metadata={'source': 'pronto/README.md'}), Document(page_content='Creating\n\nConnections are created by hovering the "+" that appears below selected nodes. There are two types of connections.', metadata={'source': 'pronto/README.md'})]
Thought:I have found the relevant information on ho

KeyboardInterrupt: 

## Next steps

1. What more context/information should be added to the documentation?
2. Evaluate the questions answers.
3. Write a testing framework that takes a question and checks if certain keywords are included.